In [ ]:
if True:
    %matplotlib notebook
elif False:
    %matplotlib inline
else:
    %matplotlib qt

In [ ]:
import matplotlib.pylab as plt
plt.rcParams['figure.figsize'] = (9, 5)

In [ ]:
from pyelegant.jupygui import twi

# Calculate Twiss

In [ ]:
twi.show_calc_gui()

# Plot Twiss

In [ ]:
twi.show_plot_gui()

# Custom Data Visualization

In [ ]:
twi_arrays = twi.DATA['twi']['arrays']

plt.figure()
plt.plot(twi_arrays['s'], twi_arrays['betax'], '.-')